### Instrucciones generales <a class="tocSkip"></a>
1. Forme un grupo de **máximo dos estudiantes**
1. Copie este notebook y **modifique el nombre de archivo** agregando los apellidos de ambos miembros. Por ejemplo si los miembros del grupo son Pablo Huijse y Ana Nuñez el nombre sería: `info147_tarea2_huijse_nuñez.ipynb`
1. Versione su trabajo usando un **repositorio privado de github**. Agregue a su compañero y a su profesor (usuario github: phuijse) en la pestaña Puede agregar a su compañero en la pestaña *Settings/Manage access*
1. Se evaluará el último commit antes de la fecha y hora de entrega
1. [Sean honestos](https://www.acm.org/about-acm/code-of-ethics-in-spanish)

# Tarea 3 Parte A: Optimización de una rutina

Considere las siguientes funciones y resuelva las actividades que se describen a continuación

In [ ]:
import numpy as np

# Generar un conjunto de datos de ND datos cada uno de largo NT con distribución log-normal
def generate_data(ND, NT):
    time = np.linspace(0, 1, num=NT)
    cov = 0.1*np.exp(-0.5*(time[:, None]  - time[:, None].T)**2/0.5**2)
    data = np.exp(np.random.multivariate_normal(mean=np.zeros(NT), cov=cov, size=ND))
    return time, data

# Ajustar un modelo polinomial de cuarto grado a los datos y calcular el MSE
def slow_function(time, data):
    ND, NT = data.shape
    X = np.vstack([time**k for k in range(4)]).T
    Phi = np.linalg.pinv(X)
    mse = np.zeros(shape=(ND,))
    theta = np.zeros(shape=(ND, 4))
    for i, y in enumerate(data):
        y_log = np.log(y)
        y_mean = np.mean(y_log)
        y_var = np.var(y_log)
        y_norm = (y_log - y_mean)/np.sqrt(y_var)
        theta[i, :] = np.dot(Phi, y_norm)
        model = np.dot(X, theta[i, :])
        mse[i] = np.mean((y_norm - model)**2)
    return mse, theta  

## Midiendo tiempo total

1. Para 20 valores de ND distintos generados con `np.logspace(1, 5, num=20).astype(int)`
    1. Genere un conjunto de datos de tamaño ND y largo NT=1000 usando `generate_data`
    1. Mida y guarde el tiempo total promedio (10 repeticiones) que toma ajustar un modelo polinomial a los ND datos con `slow_function`
        > HINT: Puede usar el argumento `-o` de la magia timeit para guardar el resultado
1. Use matplotlib para generar un gráfico de (tiempo total promedio) con barras de error y otro de (tiempo total promedio)/ND
    1. Estudie ambos gráficos y discuta lo que observa. ¿Qué está ocurriendo en el segundo gráfico? ¿Qué relación tiene con el overhead?

## Profiling 

Genere un conjunto de datos de tamaño 10000 y largo 1000

1. Haga un profiling con cProfile con la magia `%prun`
    1. Use los argumentos `-q -T texto` para escribir un archivo de texto con el resultado
    1. Imprima el resultado con funciones de `bash`
    1. ¿Cuáles son las 5 funciones con mayor tiempo total?
    1. ¿Cuáles son las 5 funciones con mayor tiempo acumulado?
1. Haga un profiling linea a linea con la magia `%lprun`
    1. Use el argumento `-T texto` para escribir un archivo de texto con el resultado
    1. Imprima el resultado con funciones de `bash`
    1. ¿Cuáles son las 5 lineas más costosas?


## Cythonizando

Escriba una función de Cython que retorne el mismo resultado que `slow_function`
1. Considere los resultados de su profiling para escribir versiones cythonizadas de las funciones de numpy que considere necesario
1. Importe el logaritmo y la raiz cuadrada de `math.h`
1. Use decoradores para levantar las verificaciones de Python
1. Use vistas para los arreglos de NumPy
1. Mida el tiempo total promedio (10 repeticiones) para conjuntos de datos de tamaño`N=np.logspace(1, 5, num=20).astype(int)` y largo 1000
1. Haga una gráfica de speed-up (tiempo cython/tiempo slow_function) en función de $N$
1. Estudie y discuta lo que observa

## Paralelizando

Escriba una versión paralela de su código Cython usando `parallel for` de OpenMP
1. Levante el GIL donde corresponda
1. Use un número de hilos adecuado para su computador
1. Mida el tiempo total promedio (10 repeticiones) para conjuntos de datos de tamaño`N=np.logspace(1, 5, num=20).astype(int)` y largo 1000
1. Haga una gráfica de speed-up (tiempo cython paralelo/tiempo cython secuencial) en función de $N$
1. Estudie y discuta lo que observa 

# Tarea 3 Parte B